In [3]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [4]:
import tensorflow as tf

from keras.models import load_model

from woundSegmentation.models.deeplab import relu6, DepthwiseConv2D, BilinearUpsampling
from woundSegmentation.utils.learning.metrics import dice_coef, precision, recall
from woundSegmentation.utils.io.data import DataGen, load_test_images, save_results

import os

import numpy as np

from woundSegmentation.models.SegNet import SegNet

import tensorflow_model_optimization as tfmot

import time
%load_ext tensorboard

In [2]:
# checa por GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print("Numero de GPUs disponiveis: ", len(tf.config.list_physical_devices('GPU')))

Numero de GPUs disponiveis:  0


In [6]:
# Insira o modelo desejado
modelName = "mobilenetv2"

# Se o modelo é podado ou não
prune = True

In [7]:
# Modelos disponiveis
models = ["fcn", "mobilenetv2", "segnet", "unet"]

# Seleção do caminho
if prune:
    dirPath = '../woundSegmentation/results_prune/'
else:
    dirPath = '../woundSegmentation/results/'

modelDirPath = dirPath + modelName + "/"

# dataset
dataset = 'Foot Ulcer Segmentation Challenge'

In [8]:
# tamanho das imagens de entrada
input_dim_x = 224
input_dim_y = 224

In [9]:
# Configuracao
if prune:
    weight_file_name = modelName + 'prunedmodelfile.hdf5'
else:
    weight_file_name = modelName + 'modelfile.hdf5'
    
color_space = 'rgb'
pred_save_path = modelDirPath + 'datapredict/'
path = '../woundSegmentation/data/' + dataset + '/'
data_gen = DataGen(path, split_ratio=0.0, x=input_dim_x, y=input_dim_y, color_space=color_space)
x_test, test_label_filenames_list = load_test_images(path)

In [10]:
# Carrega o modelo
model = load_model(modelDirPath + 'training_history/' + weight_file_name
               , custom_objects={'recall':recall,
                                 'precision':precision,
                                 'dice_coef': dice_coef,
                                 'relu6':relu6,
                                 'DepthwiseConv2D':DepthwiseConv2D,
                                 'BilinearUpsampling':BilinearUpsampling})

In [11]:
# Inicia a predição
start = time.time()
for image_batch, label_batch in data_gen.generate_data(batch_size=len(x_test), test = True):
    prediction = model.predict(image_batch, verbose=1)
    save_results(prediction, 'rgb', pred_save_path, test_label_filenames_list)
    break
end = time.time()
file = open(modelDirPath + modelName +"TimePred.txt", "w")
file.write("\n")
file.write("Predicao\n")
file.write(str(end-start))
file.close()

7/7 [==============================] - 10s 1s/step


In [12]:
print(pred_save_path)

../woundSegmentation/results_prune/mobilenetv2/datapredict/
